### EECS 730 Project - 4

#### Steps followed
1. 
2. 
3. 
4. 
5. 
6. 

#### Instructions to run
1. 
2. 
3. 
4. 
5. 

#### Import relevant packages

In [53]:
# import packages
import os
import Bio
from Bio import SeqIO
from Bio.Seq import Seq
from Bio import Align
import numpy as np
import pandas as pd
import math
#import dask.dataframe as dd
#from dask.multiprocessing import get
#from pydna.common_sub_strings import terminal_overlap
#from pydna.assembly import Assembly
#from pydna.dseqrecord import Dseqrecord

# Print versions
print('The Biopython version is {}..'.format(Bio.__version__))

The Biopython version is 1.78..


#### Create the paths for reference files

In [2]:
# Take file names as input. # Change the file name accordingly.
ffasta = 'AAA.fa' 
fhmm = 'AAA.hmm'

# Set the local paths for data
path = r'C:\Users\pmspr\Documents\HS\MS\Sem 4\EECS 730\Bioinformatics\Project 4\Docs'
fasta = os.path.join(path, ffasta) 
hmm   = os.path.join(path, fhmm)

In [102]:
#Read hmm file in to dataframes
states = [];tran_cols = [];
match = []; insert = []; trans = [];
hmmoffset = 0; holdstate = ' ';
state_cnt = 0;lineoffset = 0;

with open(hmm) as flhmm:
    for line in flhmm:
        if (line.strip()[0:3] == '//'):
            break
            
        if (line[0:4].strip() == 'HMM'):
            states = [s for s in line.split(' ') if (s not in ['','HMM','\n'])]
            state_cnt = len(states)
            hmmoffset += 1
            continue 
        if (hmmoffset == 1):
            tran_cols = [s for s in line.strip('\n').split(' ') if (s not in ['','\n'])]
            hmmoffset += 1
            continue
        if (hmmoffset == 2):
            lineoffset += 1
            if (lineoffset == 1):
                match_row = []
                match_row = [s for s in line.strip('\n').split(' ') if (s not in ['','\n'])][0:state_cnt+1]
                holdstate = match_row[0]
                match.append(match_row)
            if (lineoffset == 2):
                insert.append([holdstate]+[s for s in line.strip('\n').split(' ') if (s not in ['','\n'])][0:state_cnt+1])
            if (lineoffset == 3):
                trans.append([holdstate]+[s for s in line.strip('\n').split(' ') if (s not in ['','\n'])][0:len(tran_cols)+1])
                lineoffset = 0; holdstate = ' ';
            
match_df = pd.DataFrame(match, columns=['state']+states); match_df['state'].iloc[0] = 0;
insert_df = pd.DataFrame(insert, columns=['state']+states); insert_df['state'].iloc[0] = 0;
trans_df =  pd.DataFrame(trans,columns=['state']+tran_cols); trans_df['state'].iloc[0] = 0;
trans_df.loc[trans_df['state'] == 0] = 1;

# Display sample dataframes
display(match_df.head())
display(insert_df.tail())
display(trans_df.head())

,state,A,C,D,E,F,G,H,I,K,...,M,N,P,Q,R,S,T,V,W,Y
0,0,2.52604,4.42416,2.95323,2.65774,3.30769,2.77257,3.88316,2.70171,2.68015,...,3.63090,3.12983,3.35132,3.16734,2.88091,2.66110,2.77599,2.64066,5.09582,3.81495
1,1,2.67876,4.02341,5.04584,4.43081,3.28407,4.25179,4.58598,1.96692,4.21589,...,2.24040,4.41437,3.96304,4.34710,4.22410,3.41343,3.21354,1.53323,5.07918,3.39642
2,2,3.08766,3.31586,5.08732,4.47351,3.39603,3.97708,4.63574,2.19488,4.26134,...,3.15016,4.46297,4.65996,4.38884,4.27052,3.61471,3.35994,2.50009,5.12115,2.76043
3,3,3.45894,4.80484,5.41061,4.80074,2.11887,4.65865,3.72880,2.36028,4.59943,...,3.25424,4.81739,4.96559,4.67891,4.59615,3.97997,3.68534,2.05256,5.37563,4.23707
4,4,2.64322,3.99168,4.60427,3.48960,2.77546,4.03531,2.66459,2.49429,3.65970,...,3.64256,3.87334,4.55149,3.92136,4.03884,2.44837,2.80149,2.27618,3.94482,1.75167


,state,A,C,D,E,F,G,H,I,K,...,M,N,P,Q,R,S,T,V,W,Y
128,128,2.68618,4.42225,2.77519,2.73123,3.46354,2.40513,3.72494,3.29354,2.67741,...,4.24690,2.90347,2.73739,3.18146,2.89801,2.37887,2.77519,2.98518,4.58477,3.61503
129,129,2.68618,4.42225,2.77519,2.73123,3.46354,2.40513,3.72494,3.29354,2.67741,...,4.24690,2.90347,2.73739,3.18146,2.89801,2.37887,2.77519,2.98518,4.58477,3.61503
130,130,2.68618,4.42225,2.77519,2.73123,3.46354,2.40513,3.72494,3.29354,2.67741,...,4.24690,2.90347,2.73739,3.18146,2.89801,2.37887,2.77519,2.98518,4.58477,3.61503
131,131,2.68618,4.42225,2.77519,2.73123,3.46354,2.40513,3.72494,3.29354,2.67741,...,4.24690,2.90347,2.73739,3.18146,2.89801,2.37887,2.77519,2.98518,4.58477,3.61503
132,132,2.68618,4.42225,2.77519,2.73123,3.46354,2.40513,3.72494,3.29354,2.67741,...,4.24690,2.90347,2.73739,3.18146,2.89801,2.37887,2.77519,2.98518,4.58477,3.61503


,state,m->m,m->i,m->d,i->m,i->i,d->m,d->d
0,1,1,1,1,1,1,1,1
1,1,0.00734,5.31361,6.03596,0.61958,0.77255,0.48576,0.95510
2,2,0.00734,5.31361,6.03596,0.61958,0.77255,0.48576,0.95510
3,3,0.00734,5.31361,6.03596,0.61958,0.77255,0.48576,0.95510
4,4,0.00734,5.31361,6.03596,0.61958,0.77255,0.48576,0.95510


In [103]:
# Read the sequence file
seql = []
with open(fasta) as flfst:
    for line in flfst:
        if (line.strip()[0] != '>'):
            seq = line.strip('\n')
seql = [seq[s] for s in range(len(seq))]
print('Input sequence with length {}:\n{}'.format(len(seq),seq))

Input sequence with length 130:
VLFEGPPGTGKTSCARVIANQAGIPLLYVPLEAVMSKYYGESERLLGAVFSQANELPDGAIIFLDEIDAFAISRDSEMHEATRRVLSVLLRQIDGFEQEKKVVVIAATNRKQDLDPALISRFDSMIMFDL


In [120]:
# Calculate matrices
ncols = match_df.shape[0] - 1     #Ignore compo
nrows = len(seq) 

# Declare and initialize

# Match matrix
vm = np.zeros(shape=(nrows+1,ncols+1),dtype=np.float64)
vm[0,0] = 0; vm[0,1:] = -1000; vm[1:,0] = -1000

# Insert matrix
vi = np.zeros(shape=(nrows+1,ncols+1),dtype=np.float64)
vi[0:,0] = -1000

# Delete matrix
vd = np.array([0,0], dtype=np.float64)
vd = np.zeros(shape=(nrows+1,ncols+1),dtype=np.float64)
vd[0,0:] = -1000

# Terminate matrix
vt = np.zeros(shape=(nrows+1,ncols+1),dtype=np.float64)

# Trace matrix
tb = np.array([['x' for i in range(0,ncols+1)]for j in range(0,nrows+1)], dtype=np.object)

# Fill the matrice
n = 0.0; n1 = 0.0; n2 = 0.0; n3 = 0.0;
for j in range(1, nrows+1):
    for i in range(1,ncols+1):
        
        # Match matrix
        if (trans_df['m->m'].iloc[j-1] == '*'):
            trans_df['m->m'].iloc[j-1] = 1
        
        if (trans_df['i->m'].iloc[j-1] == '*'):
            trans_df['i->m'].iloc[j-1] = 1
        
        if (trans_df['d->m'].iloc[j-1] == '*'):
            trans_df['d->m'].iloc[j-1] = 1
            
        n1 = vm[j-1,i-1] + math.log10(float(trans_df['m->m'].iloc[j-1]))
        n2 = vi[j-1,i-1] + math.log10(float(trans_df['i->m'].iloc[j-1]))
        n3 = vd[j-1,i-1] + math.log10(float(trans_df['d->m'].iloc[j-1]))
        n  = math.log10(float(match_df[seq[j-1]].iloc[i]) / seql.count(seq[j-1]))
        vm[j,i] = n + np.amax(np.array([n1,n2,n3],dtype=np.float64))
        
        # Insert matrix
        if (trans_df['m->i'].iloc[j] == '*'):
            trans_df['m->i'].iloc[j] = 1
        
        if (trans_df['i->i'].iloc[j] == '*'):
            trans_df['i->i'].iloc[j] = 1
            
        n1 = vm[j,i-1] + math.log10(float(trans_df['m->i'].iloc[j]))
        n2 = vi[j,i-1] + math.log10(float(trans_df['i->i'].iloc[j]))
        n3 = vd[j,i-1]
        n  = math.log10(float(insert_df[seq[j-1]].iloc[i]) / seql.count(seq[j-1]))
        vi[j,i] = n + np.amax(np.array([n1,n2,n3],dtype=np.float64))
        
        # Delete matrix
        if (trans_df['m->d'].iloc[j] == '*'):
            trans_df['m->d'].iloc[j] = 1
        
        if (trans_df['d->d'].iloc[j] == '*'):
            trans_df['d->d'].iloc[j] = 1
            
        n1 = vm[j-1,i] + math.log10(float(trans_df['m->d'].iloc[j])) 
        n2 = vi[j-1,i] 
        n3 = vd[j-1,i] + math.log10(float(trans_df['d->d'].iloc[j]))
        vd[j,i] = np.amax(np.array([n1,n2,n3],dtype=np.float64))
        
        # Termination matrix
        maxar = np.array([vm[j,i],vi[j,i],vd[j,i]],dtype=np.float64)
        vt[j,i] = np.amax(maxar)
        
        # Traceback matrix
        if(np.argmax(maxar) == 0):
            tb[j,i] = 'M'
        if(np.argmax(maxar) == 1):
            tb[j,i] = 'I'
        if(np.argmax(maxar) == 2):
            tb[j,i] = 'D'
            
print(tb)

[['x' 'x' 'x' ... 'x' 'x' 'x']
 ['x' 'D' 'D' ... 'D' 'D' 'D']
 ['x' 'D' 'D' ... 'D' 'D' 'D']
 ...
 ['x' 'D' 'D' ... 'D' 'D' 'D']
 ['x' 'D' 'D' ... 'D' 'D' 'D']
 ['x' 'D' 'D' ... 'D' 'D' 'D']]


#### Main Logic

In [3]:
# Determine the lengths
ls0 = len(seq[0]); ncols = ls0 + 1
ls1 = len(seq[1]); nrows = ls1 + 1

# Create numpy arrays from the strings
s0_arr = np.array(['*' for i in range(0, ncols)], dtype=np.object)
s0_arr[1:] = [seq[0][i] for i in range(0, ls0)]
#print(s0_arr)

s1_arr = np.array(['*' for i in range(0, nrows)], dtype=np.object)
s1_arr[1:] = [seq[1][i] for i in range(0, ls1)]
#print(s1_arr)

# Define the scoring parameters
match_score = 1.0
mismatch_score = -2.0
gap_open = -5
gap_extn = -1

# Define the upper layer matrix and initialize
# Initialize first column = gap_open + row_index * gap_extn
upp_arr = np.zeros(shape=(nrows,ncols),dtype=np.int64)
upp_arr[0,0] = -1000
upp_arr[1:,0] = np.array([gap_open + i*gap_extn for i in range(1,nrows)],dtype=np.int64)
#print(upp_arr)

# Define the left layer matrix and initialize
# Initialize first row = gap_open + col_index * gap_extn
lft_arr = np.zeros(shape=(nrows,ncols),dtype=np.int64)
lft_arr[0,0] = -1000
lft_arr[0,1:] = np.array([gap_open + i*gap_extn for i in range(1,ncols)],dtype=np.int64)
#print(lft_arr)

# Define the diagonal layer matrix and initialize
# Initialize first column, row = 0
dig_arr = np.zeros(shape=(nrows,ncols),dtype=np.int64)
dig_arr[0,0] = -1000
#print(dig_arr)

# Define the diagonal layer matrix and initialize
# Initialize first column = gap_open + row_index * gap_extn
# Initialize first row = gap_open + col_index * gap_extn
sub_arr = np.zeros(shape=(nrows,ncols),dtype=np.int64)
sub_arr[0,1:] = np.array([gap_open + i*gap_extn for i in range(1,ncols)],dtype=np.int64)
sub_arr[1:,0] = np.array([gap_open + i*gap_extn for i in range(1,nrows)],dtype=np.int64)
#print(sub_arr)


# Define trace back matrix
# Initialize first column = 'u'
# Initialize first row = 'l'
trace_arr = np.array([['x' for i in range(0,ncols)]for j in range(0,nrows)], dtype=np.object)
trace_arr[0,1:] = np.array(['l' for i in range(0,ls0)],dtype=np.object)
trace_arr[1:,0] = np.array(['u' for i in range(0,ls1)],dtype=np.object)

In [4]:
# Fill the intermediate arrays
neighbors = np.array([0,0], dtype=np.int64)
fneigh = np.array([0,0,0], dtype=np.int64)
for i in range(1,nrows):
    for j in range(1,ncols):
        
        # Fill upper matrix
        neighbors = [0,0]
        neighbors[0] = sub_arr[i,j-1] + gap_open + gap_extn
        neighbors[1] = upp_arr[i,j-1] + gap_extn
        upp_arr[i,j] = np.amax(neighbors)
        
        # Fill left matrix
        neighbors = [0,0]
        neighbors[0] = sub_arr[i-1,j] + gap_open + gap_extn
        neighbors[1] = lft_arr[i-1,j] + gap_extn
        lft_arr[i,j] = np.amax(neighbors)
        
        # fill diagonal matrix
        if(s1_arr[i] == s0_arr[j]):
            dig_arr[i,j] = sub_arr[i-1,j-1] + match_score
        if(s1_arr[i] != s0_arr[j]):
            dig_arr[i,j] = sub_arr[i-1,j-1] + mismatch_score
        
        # fill traceback matrix
        fneigh = [0,0,0]
        fneigh = np.array([dig_arr[i,j],upp_arr[i,j],lft_arr[i,j]],dtype=np.int64)
        sub_arr[i,j] = np.amax(fneigh)
        if(np.argmax(fneigh) == 0):
            trace_arr[i,j] = 'd'
        if(np.argmax(fneigh) == 1):
            trace_arr[i,j] = 'l'
        if(np.argmax(fneigh) == 2):
            trace_arr[i,j] = 'u'

print('Upper layer Matrix:')
print(upp_arr)
print()

print('Left layer Matrix:')
print(lft_arr)
print()

print('Diagonal layer Matrix:')
print(dig_arr)
print()

print('Substitution Matrix:')
print(sub_arr)
print()

print('Trace back Matrix:')
print(trace_arr)

Upper layer Matrix:
[[-1000     0     0 ...     0     0     0]
 [   -6    -7    -5 ...  -289  -290  -291]
 [   -7    -8    -9 ...  -287  -288  -289]
 ...
 [ -271  -272  -273 ...   223   230   229]
 [ -272  -273  -274 ...   222   224   231]
 [ -273  -274  -275 ...   221   223   225]]

Left layer Matrix:
[[-1000    -6    -7 ...  -291  -292  -293]
 [    0    -7    -8 ...  -292  -293  -294]
 [    0    -5    -9 ...  -293  -294  -295]
 ...
 [    0  -269  -267 ...   223   222   221]
 [    0  -270  -268 ...   230   224   223]
 [    0  -271  -269 ...   229   231   225]]

Diagonal layer Matrix:
[[-1000     0     0 ...     0     0     0]
 [    0     1    -8 ...  -292  -293  -294]
 [    0    -8     2 ...  -287  -288  -289]
 ...
 [    0  -272  -267 ...   236   230   229]
 [    0  -273  -268 ...   230   237   231]
 [    0  -274  -269 ...   229   231   238]]

Substitution Matrix:
[[   0   -6   -7 ... -291 -292 -293]
 [  -6    1   -5 ... -289 -290 -291]
 [  -7   -5    2 ... -287 -288 -289]
 ...
 [-271

In [5]:
# Determine the alignment using traceback matrix
i = trace_arr.shape[0] - 1
j = trace_arr.shape[1] - 1
a0 = []; a1 = []; a2 = [];
score = 0;ugap_ind = 0; lgap_ind = 0;

# Traverse from right bottom to the top left to create the alignment
while(trace_arr[i,j] != 'x'):
    if(trace_arr[i,j] == 'd'):
        a0.append(s0_arr[j])
        a1.append(s1_arr[i])
        if(s0_arr[j] == s1_arr[i]):
            a2.append('|')
            score = score + match_score
            
        if(s0_arr[j] != s1_arr[i]):
            a2.append('*')
            score = score + mismatch_score
        i = i-1;j=j-1;
    if(trace_arr[i,j] == 'l'):
        a0.append(s0_arr[j])
        a1.append('-')
        a2.append(' ')
        lgap_ind = lgap_ind + 1
        score = score + gap_open + lgap_ind*gap_extn
        i=i;j=j-1;
    if(trace_arr[i,j] == 'u'):
        a0.append('-')
        a1.append(s1_arr[i])
        a2.append(' ')
        ugap_ind = ugap_ind + 1
        score = score + gap_open + ugap_ind*gap_extn
        i=i-1;j=j;
    
as0 = ''.join(a0[::-1]); as1 = ''.join(a1[::-1]); as2 = ''.join(a2[::-1])

# print the output to the console
print()
print('Global Alignment without Affine gap with score = {}:'.format(score))
print(as0)
print(as2)
print(as1)


Global Alignment without Affine gap with score = -42.0:
ACTCTTCTGGTCCCCACAGACTCAGAGAGAACCCACCATGGTGCTGTCTCCTGCCGACAAGACCAACGTCAAGGCCGCCTGGGGTAAGGTCGGCGCGCACGCTGGCGAGTATGGTGCGGAGGCCCTGGAGAGGTGAGGCTCCCTCCCCTGCTCCGACCCGGGCTCCTCGCCCGCCCGGACCCACAGGCCACCCTCAACCGTCCTGGCCCCGGACCCAAACCCCACCCCTCACTCTGCTTCTCCCCGCAGGATGTTCCTGTCCTTCCCCACCACCAAGACCTACTTCCC
|||||||||||||||||||||||    ||      ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||          |||||||||||||||||||||||||||||||||||||||||||||||
ACTCTTCTGGTCCCCACAGACTC----AG------CCATGGTGCTGTCTCCTGCCGACAAGACCAACGTCAAGGCCGCCTGGGGTAAGGTCGGCGCGCACGCTGGCGAGTATGGTGCGGAGGCCCTGGAGAGGTGAGGCTCCCTCCCCTGCTCCGACCCGGGCTCCTCGCCCGCCCGGACCCACAGGCCACCCTCAACCGTCCTGGCCCCGGACCCAAACCCCACCCCTCA----------CCCCGCAGGATGTTCCTGTCCTTCCCCACCACCAAGACCTACTTCCC


#### Testing against biopython 

In [6]:
# Sequence alignment with affine gap
aligner_affine = Align.PairwiseAligner()

aligner_affine.match_score = 1.0
aligner_affine.mismatch_score = -2.0
aligner_affine.open_gap_score = -5
aligner_affine.extend_gap_score = -1
alignments = aligner_affine.align(seq[0], seq[1])
for al in alignments:
    print('Below Alignment has a score:{}'.format(al.score))
    print(al)

Below Alignment has a score:240.0
ACTCTTCTGGTCCCCACAGACTCAGAGAGAACCCACCATGGTGCTGTCTCCTGCCGACAAGACCAACGTCAAGGCCGCCTGGGGTAAGGTCGGCGCGCACGCTGGCGAGTATGGTGCGGAGGCCCTGGAGAGGTGAGGCTCCCTCCCCTGCTCCGACCCGGGCTCCTCGCCCGCCCGGACCCACAGGCCACCCTCAACCGTCCTGGCCCCGGACCCAAACCCCACCCCTCACTCTGCTTCTCCCCGCAGGATGTTCCTGTCCTTCCCCACCACCAAGACCTACTTCCC
|||||||||||||||||||||||||----------||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||----------|||||||||||||||||||||||||||||||||||||||||||||||
ACTCTTCTGGTCCCCACAGACTCAG----------CCATGGTGCTGTCTCCTGCCGACAAGACCAACGTCAAGGCCGCCTGGGGTAAGGTCGGCGCGCACGCTGGCGAGTATGGTGCGGAGGCCCTGGAGAGGTGAGGCTCCCTCCCCTGCTCCGACCCGGGCTCCTCGCCCGCCCGGACCCACAGGCCACCCTCAACCGTCCTGGCCCCGGACCCAAACCCCACCCCTCA----------CCCCGCAGGATGTTCCTGTCCTTCCCCACCACCAAGACCTACTTCCC

Below Alignment has a score:240.0
ACTCTTCTGGTCCCCACAGACTCAGAGAGAACCCACCATGGTGCTGTCTCCTGCCGACAAGACC